# 5교시 집계 연산

### 목차
* [1. 집계 함수](#1.-집계-함수)
* [2. 그룹 함수](#2.-그룹-함수)
* [참고자료](#참고자료)


In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)
# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

In [2]:
""" 구매 이력 데이터 """
df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/retail-data/all")
    .coalesce(5)
)
df.cache()
df.createOrReplaceTempView("dfTable")

In [3]:
df.show(5, truncate=False)
df.count()

+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |12/1/2010 8:26|2.55     |17850     |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |12/1/2010 8:26|2.75     |17850     |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
+---------+---------+-----------------------------------

541909

## 1. 집계 함수
### 1.1 로우 수 (count, countDistinct, approx_count_distinct)

In [7]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [8]:
from pyspark.sql.functions import *

df.selectExpr("count(*)").show()
df.where("Description is null").selectExpr("count(1)").show() # 1,454
df.selectExpr("count(Description)").show() # 540,455 + 1,454 = 541,909

+--------+
|count(1)|
+--------+
|  541909|
+--------+

+--------+
|count(1)|
+--------+
|    1454|
+--------+

+------------------+
|count(Description)|
+------------------+
|            540455|
+------------------+



In [5]:
from pyspark.sql.functions import *
# 명시적으로 컬럼을 지정한 경우 해당 컬럼이 널 인 경우 해당 로우는 제외됩니다
df.select(countDistinct("StockCode")).show()
df.selectExpr("count(distinct StockCode)").show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



In [6]:
from pyspark.sql.functions import *
# 근사치로 구하지만 연산 속도가 빠름
df.select(approx_count_distinct("StockCode", 0.1)).show() # 0.1은 최대 추정 오류율
df.select(approx_count_distinct("StockCode", 0.01)).show() 

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            4079|
+--------------------------------+



### 1.2 수치 집계 함수 (first, last, min, max, sum, sumDistinct, avg)

In [5]:
from pyspark.sql.functions import *
df.select(first("StockCode"), last("StockCode")).show(1) # null도 감안하려면 True

df.select(min("Quantity"), max("Quantity")).show(1)
df.select(min("Description"), max("Description")).show(1) # 문자열

df.select(sum("Quantity")).show(1)
df.select(sumDistinct("Quantity")).show(1) # 고유값을 합산

+-----------------------+----------------------+
|first(StockCode, false)|last(StockCode, false)|
+-----------------------+----------------------+
|                 85123A|                 22138|
+-----------------------+----------------------+

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+

+--------------------+-----------------+
|    min(Description)| max(Description)|
+--------------------+-----------------+
| 4 PURPLE FLOCK D...|wrongly sold sets|
+--------------------+-----------------+

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



### 1.3 통계 집계 함수 (avg, mean, variance, stddev) 
* 표본표준분산 및 편차: variance, stddev
* 모표준분산 및 편차 : var_pop, stddev_pop

In [9]:
from pyspark.sql.functions import *

df.select(
    count("Quantity").alias("total_transcations"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_transcations"),    
).selectExpr(
    "total_purchases / total_transcations",
    "avg_purchases",
    "mean_transcations").show(3)

+--------------------------------------+----------------+-----------------+
|(total_purchases / total_transcations)|   avg_purchases|mean_transcations|
+--------------------------------------+----------------+-----------------+
|                      9.55224954743324|9.55224954743324| 9.55224954743324|
+--------------------------------------+----------------+-----------------+



In [10]:
df.select(
    variance("Quantity")
    , stddev("Quantity")
    , var_samp("Quantity")
    , stddev_samp("Quantity")
    , var_pop("Quantity")
    , stddev_pop("Quantity")
).show()

+------------------+---------------------+------------------+---------------------+------------------+--------------------+
|var_samp(Quantity)|stddev_samp(Quantity)|var_samp(Quantity)|stddev_samp(Quantity)| var_pop(Quantity)|stddev_pop(Quantity)|
+------------------+---------------------+------------------+---------------------+------------------+--------------------+
|47559.391409298696|   218.08115785023404|47559.391409298696|   218.08115785023404|47559.303646609005|  218.08095663447784|
+------------------+---------------------+------------------+---------------------+------------------+--------------------+



## 2. 그룹 함수

### 2.1 표현식을 이용한 그룹화

In [7]:
from pyspark.sql.functions import count
df.printSchema()
df.groupBy("InvoiceNo", "CustomerId").agg(expr("count(Quantity) as CountOfQuantity")).show(5)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

+---------+----------+---------------+
|InvoiceNo|CustomerId|CountOfQuantity|
+---------+----------+---------------+
|   536846|     14573|             76|
|   537026|     12395|             12|
|   537883|     14437|              5|
|   538068|     17978|             12|
|   538279|     14952|              7|
+---------+----------+---------------+
only showing top 5 rows



### 2.2 맵을 이용한 그룹화
> 파이선의 딕셔너리 데이터 타입을 활용하여 집계함수의 표현이 가능 

In [8]:
df.groupBy("InvoiceNo").agg( { "Quantity" : "avg", "UnitPrice" : "stddev_pop" } ).show(5)

+---------+---------------------+------------------+
|InvoiceNo|stddev_pop(UnitPrice)|     avg(Quantity)|
+---------+---------------------+------------------+
|   536596|    6.618375094302897|               1.5|
|   536938|   2.4313249096586267|33.142857142857146|
|   537252|                  0.0|              31.0|
|   537691|    2.761232695735729|              8.15|
|   538041|                  0.0|              30.0|
+---------+---------------------+------------------+
only showing top 5 rows



### <font color=blue>1. [중급]</font> 구매 이력 CSV "data/retail-data/all" 파일을 읽고
#### 1. 스키마를 출력하세요
#### 2. 데이터 10건을 출력하세요
#### 3. 상품코드(StockCode)의 유일한 값의 갯수를 출력하세요
#### 4. 상품단가(UnitPrice)의 최소, 최대 값을 출력하세요
#### 5. 송장번호(StockCode)별로 송장별총매출금액(TotalInvoicePrice)를 계산하고 내림차순으로 정렬하세요
#### 6. 송장별총매출금액(TotalInvoicePrice)이 최고금액이 송장을 필터하여 검증해 보세요
##### 예를 들어 `select sum(unit-price * quantity) from table where invoiceno = '123456'` 와 같은 쿼리로 검증이 가능합니다

<details><summary>[실습7] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df1 = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/retail-data/all")
)
df1.printSchema()
df1.show()
answer = df1.withColumn("TotalPrice", expr("UnitPrice * Quantity")).groupBy("InvoiceNo").agg(sum("TotalPrice").alias("TotalInvoicePrice"))
answer.printSchema()
display(answer.orderBy(desc("TotalInvoicePrice")).limit(10))

df1.where("InvoiceNo = '581483'").select(sum(expr("UnitPrice * Quantity"))).show()
```

</details>


In [15]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|  

InvoiceNo,AveragePerInvoice
581483,168469.6
541431,77183.6
574941,52940.93999999999
576365,50653.909999999996
556444,38970.0
567423,31698.159999999996
556917,22775.930000000008
572209,22206.0
567381,22104.8
563614,21880.439999999995


+---------------------------+
|sum((UnitPrice * Quantity))|
+---------------------------+
|                   168469.6|
+---------------------------+



### <font color=green>2. [기본]</font> 매출 테이블 "data/tbl_purchase.csv" CSV 파일을 읽고
#### 1. 스키마를 출력하세요
#### 2. 데이터 10건을 출력하세요
#### 3. 제품(p_name)별 금액(p_amount) 의 전체 합인 총 매출금액(sum_amount)을 구하세요

<details><summary>[실습2] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df2 = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_purchase.csv")
)
df2.printSchema()
df2.show()
answer = df2.groupBy("p_name").agg(sum("p_amount").alias("sum_amount"))
answer.printSchema()
display(answer)

```

</details>


In [4]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


p_name,sum_amount
LG Gram,5300000
LG Computer,4500000
LG Cyon,1400000
LG TV,3500000
GoldStar TV,100000
LG DIOS,2000000


### <font color=green>3. [기본]</font> 매출 테이블 "data/tbl_purchase.csv" CSV 파일을 읽고
#### 1. 스키마를 출력하세요
#### 2. 데이터 10건을 출력하세요
#### 3. 구매 금액의 합이 가장 높은 고객(p_uid)을 구하세요

<details><summary>[실습3] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df3 = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_purchase.csv")
)
df3.printSchema()
df3.show()
answer = df2.groupBy("p_uid").agg(sum("p_amount").alias("sum_amount_per_user"))
answer.printSchema()
display(answer)
```

</details>


In [6]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


p_uid,sum_amount_per_user
5,6000000
4,4500000
1,3800000
2,1400000
3,1000000
0,100000


### <font color=red>4. [고급]</font> 샌프란시스코 긴급출동 데이터 CSV 파일인 "data/learning-spark/sf-fire-calls.csv"를 읽고
#### 1. 스키마를 출력하세요
#### 2. 데이터를 3건 출력하세요
#### 3. 호출의 종류(CallType)가 어떤 것들이 있는지 출력하세요 (중복제거)
#### 3. 샌프란시스코에서 발생의 가장 빈도수가 높은 종류(CallType)를 구하고 빈도수를 구하세요
#### 4. 샌프란시스코에서 발생하는 최고 빈도수 3건은 무엇인가요? 

<details><summary>[실습3] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df3 = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("data/learning-spark/sf-fire-calls.csv")
)
df3.printSchema()
df3.show(3)
df3.createOrReplaceTempView("fire_calls")
spark.sql("select distinct(CallType) from fire_calls").show(truncate=False)

answer = spark.sql("select CallType, count(CallType) as CallTypeCount from fire_calls group by CallType order by CallTypeCount desc")
display(answer.limit(3))
```

</details>


In [34]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

CallType,CallTypeCount
Medical Incident,113794
Structure Fire,23319
Alarms,19406


### <font color=red>5. [고급]</font> 샌프란시스코 긴급출동 데이터 CSV 파일인 "data/learning-spark/sf-fire-calls.csv"를 읽고 다음과 같은 질문도 실습해 보면 재미있을 것 같습니다
#### 1. 2018 년의 모든 화재 신고 유형은 무엇 이었습니까?
#### 2. 2018 년의 몇 월에 화재 신고가 가장 많았습니까?
#### 3. 샌프란시스코에서 2018 년에 가장 많은 화재 신고가 발생한 지역은 어디입니까?
#### 4. 2018 년에 화재 신고에 대한 응답 시간이 가장 나쁜 지역은 어디입니까?
#### 5. 2018 년 중 어느 주에 화재 신고가 가장 많았습니까?
#### 6. 이웃, 우편 번호, 화재 전화 건수간에 상관 관계가 있습니까?
#### 7. Parquet 파일 또는 SQL 테이블을 사용하여이 데이터를 저장하고 다시 읽을 수있는 방법은 무엇입니까?


## 참고자료

#### 1. [Spark Programming Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)
#### 2. [PySpark SQL Modules Documentation](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html)
#### 3. <a href="https://spark.apache.org/docs/3.0.1/api/sql/" target="_blank">PySpark 3.0.1 Builtin Functions</a>
#### 4. [PySpark Search](https://spark.apache.org/docs/latest/api/python/search.html)
#### 5. [Pyspark Functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?#module-pyspark.sql.functions)